In [1]:
import sys
sys.path.append('..')

import time
import os
import re
from collections import defaultdict
import pprint as pp

from tqdm import tqdm
RE_FNAME = re.compile(r'LP\-(?P<idx>[0-9])\-STRM\-[0-9]\-(?P<pair>[A-Z]+).*\.csv')

from utils import GCStorage
from constants import CREDENTIAL_PATH, ALL_DAYS

import pandas as pd

import shutil
import csv
import numpy as np
from subprocess import call
import pickle

import datetime
from dateutil.parser import parse
from pathlib import Path

from utils import GCStorage
from constants import *
from multiprocessing import Pool, Manager, Value

In [2]:
time_interval = 60
organized_path = TEMP_FOLDER / 'organized_data'

jfolder = f'joint_{time_interval}_data'
joint_path = TEMP_FOLDER / f'{jfolder}'
joint_path.mkdir(parents=True, exist_ok=True)

storage = GCStorage('FX_Trading', 'integral_data', CREDENTIAL_PATH)
candle_interval = datetime.timedelta(seconds=time_interval)

In [3]:
valid_pairs = ['EURUSD', 'GBPUSD', 'NZDUSD', 'USDCAD', 'USDCHF', 'USDJPY']
# valid_pairs = ['AUDUSD']
# valid_pairs = ['EURUSD']
valid_intervals = [10, 30, 60, 300, 600]
total_count = len(valid_pairs) * len(valid_intervals) * len(ALL_DAYS) * 5
print(total_count)

3750


In [4]:
def handled_upload(storage, local_path, cloud_path):
    counter = 0
    while counter < 5:
        try:
            storage.upload(local_path, cloud_path)
            counter = 100
        except:
            counter += 1
            if counter >= 5:
                print(f'Failed to upload {local_path}')
            time.sleep(2)

In [5]:
def generate_candles(df, start_time, increment):
    prev_bid = df.iloc[0]['bid price']
    prev_ask = df.iloc[0]['ask price']
    bid = {'Open': [], 'Close': [], 'High': [], 'Low': [], 'Volume': []}
    ask = {'Open': [], 'Close': [], 'High': [], 'Low': [], 'Volume': []}
    candle_starts = []
    df_iloc = 0
    end_time = start_time + datetime.timedelta(days=1)
    num_intervals = int((end_time - start_time) / increment)
    print(num_intervals)
    
    # for i in tqdm(range(num_intervals)):
    for i in range(num_intervals):
        candle_start = start_time + i * increment
        candle_end = start_time + (i + 1) * increment
    
        cur_bid = {'Open': prev_bid, 'Close': prev_bid, 'High': prev_bid, 'Low': prev_bid, 'Volume': 0}
        cur_ask = {'Open': prev_ask, 'Close': prev_ask, 'High': prev_ask, 'Low': prev_ask, 'Volume': 0}
        bid_opened = False
        ask_opened = False
        # print(df_iloc, candle_start)
        while df_iloc < len(df) and df.iloc[df_iloc]['time'] >= candle_start and df.iloc[df_iloc]['time'] < candle_end:
            local_bid = df.iloc[df_iloc]['bid price']
            local_ask = df.iloc[df_iloc]['ask price']
            if local_bid > 0.1:
                cur_bid['Close'] = local_bid
                cur_bid['High'] = max(local_bid, cur_bid['High'])
                cur_bid['Low'] = min(local_bid, cur_bid['Low'])
                cur_bid['Volume'] += df.iloc[df_iloc]['bid volume']
                if not bid_opened:
                    cur_bid['Open'] = local_bid
                    bid_opened = True
            
            if local_ask > 0.1:
                cur_ask['Close'] = local_ask
                cur_ask['High'] = max(local_ask, cur_ask['High'])
                cur_ask['Low'] = min(local_ask, cur_ask['Low'])
                cur_ask['Volume'] += df.iloc[df_iloc]['ask volume']
                if not ask_opened:
                    cur_ask['Open'] = local_ask
                    ask_opened = True
            
            df_iloc += 1
        
        candle_starts.append(candle_start)
        for k in cur_bid:
            bid[k].append(cur_bid[k])
            ask[k].append(cur_ask[k])
            
        prev_bid = bid['Close'][-1]
        prev_ask = ask['Close'][-1]
    
    assert len(candle_starts) == num_intervals
    for k in bid:
        assert len(bid[k]) == num_intervals
        assert len(ask[k]) == num_intervals
    
    return candle_starts, bid, ask, df_iloc

In [6]:
def generate_for_one_file(args):
    day, pair, lp, value = args
    day = day.replace('-', '')
    
    fname = TEMP_FOLDER / f'organized_data/{pair}/{day}/{day}-{pair}-{lp}.csv'
    print(fname)
    try:
        df = pd.read_csv(fname, parse_dates=['time'])
    except:
        print(f'Unable to read csv {fname}')
        return

    storage_path = TEMP_FOLDER / f'pickled_data/{pair}/{day}'
    os.makedirs(storage_path, exist_ok=True)

    for interval in valid_intervals:
        candle_path = TEMP_FOLDER / f'candle_{interval}_data/{pair}/{day}'
        os.makedirs(candle_path, exist_ok=True)

        cname = TEMP_FOLDER / f'candle_{interval}_data/{pair}/{day}/{day}-{pair}-{lp}.pickle'
        print(cname)

        candle_starts, bids, asks, last_iloc = generate_candles(df, parse(day), datetime.timedelta(seconds=interval))
        data = {'candle_starts': candle_starts,
                'candle_bids': bids,
                'candle_asks': asks}

        c_cloud = f'candle_{interval}_data/{pair}/{day}/{day}-{pair}-{lp}.pickle'
        pickle.dump(data, open(cname, 'wb'))
        # storage.upload(cname, c_cloud)
        handled_upload(storage, cname, c_cloud)

        value.value += 1
        print(f'Completed {value.value}/{total_count} candles')

    pname = TEMP_FOLDER / f'pickled_data/{pair}/{day}/{day}-{pair}-{lp}.pickle'
    p_cloud = f'pickled_data/{pair}/{day}/{day}-{pair}-{lp}.pickle'
    print(pname)
    df = df.iloc[:last_iloc]  # Truncate to only the useful part

    pickle.dump(df, open(pname, 'wb'))
    # storage.upload(pname, p_cloud)
    handled_upload(storage, pname, p_cloud)

In [8]:
manager = Manager()

completed_count = manager.Value('completed', 0)
pl = Pool(72)

all_arguments = []

/home/jingbo/temp_store/organized_data/EURUSD/20190201/20190201-EURUSD-2.csv
/home/jingbo/temp_store/organized_data/AUDUSD/20190214/20190214-AUDUSD-1.csv
/home/jingbo/temp_store/organized_data/AUDUSD/20190214/20190214-AUDUSD-4.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190205/20190205-EURUSD-1.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190201/20190201-EURUSD-5.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190205/20190205-EURUSD-4.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190204/20190204-EURUSD-3.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190206/20190206-EURUSD-2.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190207/20190207-EURUSD-3.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190206/20190206-EURUSD-5.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190211/20190211-EURUSD-2.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190208/20190208-EURUSD-1.csv
/home/jingbo/temp_store/organized_data/EURUSD/20190208/20190208-EURUSD-4.csv

8640
/home/jingbo/temp_store/candle_10_data/EURUSD/20190214/20190214-EURUSD-5.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190301/20190301-GBPUSD-3.pickle
8640
/home/jingbo/temp_store/candle_10_data/EURUSD/20190219/20190219-EURUSD-1.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190215/20190215-GBPUSD-5.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190227/20190227-GBPUSD-4.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190219/20190219-GBPUSD-4.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190220/20190220-GBPUSD-5.pickle
8640
/home/jingbo/temp_store/candle_10_data/EURUSD/20190227/20190227-EURUSD-1.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190228/20190228-GBPUSD-5.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190213/20190213-GBPUSD-3.pickle
8640
/home/jingbo/temp_store/candle_10_data/EURUSD/20190228/20190228-EURUSD-5.pickle
8640
/home/jingbo/temp_store/candle_10_data/GBPUSD/20190211/20190

/home/jingbo/temp_store/candle_30_data/GBPUSD/20190301/20190301-GBPUSD-3.pickle
2880
Completed 45/3750 candles
/home/jingbo/temp_store/candle_30_data/EURUSD/20190213/20190213-EURUSD-1.pickle
2880
Completed 46/3750 candles
/home/jingbo/temp_store/candle_60_data/NZDUSD/20190201/20190201-NZDUSD-1.pickle
1440
Completed 47/3750 candles
/home/jingbo/temp_store/candle_60_data/EURUSD/20190227/20190227-EURUSD-4.pickle
1440
Completed 48/3750 candles
/home/jingbo/temp_store/candle_60_data/EURUSD/20190213/20190213-EURUSD-4.pickle
1440
Completed 49/3750 candles
/home/jingbo/temp_store/candle_30_data/EURUSD/20190225/20190225-EURUSD-2.pickle
2880
Completed 50/3750 candles
/home/jingbo/temp_store/candle_60_data/EURUSD/20190225/20190225-EURUSD-5.pickle
1440
Completed 51/3750 candles
/home/jingbo/temp_store/candle_60_data/EURUSD/20190222/20190222-EURUSD-4.pickle
1440
Completed 52/3750 candles
/home/jingbo/temp_store/candle_60_data/NZDUSD/20190201/20190201-NZDUSD-4.pickle
1440
Completed 53/3750 candles
/

1440
Completed 118/3750 candles
/home/jingbo/temp_store/candle_300_data/EURUSD/20190206/20190206-EURUSD-5.pickle
288
Completed 119/3750 candles
/home/jingbo/temp_store/candle_600_data/EURUSD/20190208/20190208-EURUSD-4.pickle
144
Completed 120/3750 candles
/home/jingbo/temp_store/candle_600_data/NZDUSD/20190201/20190201-NZDUSD-1.pickle
144
Completed 121/3750 candles
/home/jingbo/temp_store/candle_60_data/EURUSD/20190205/20190205-EURUSD-1.pickle
1440
Completed 122/3750 candles
/home/jingbo/temp_store/candle_300_data/GBPUSD/20190204/20190204-GBPUSD-5.pickle
288
Completed 123/3750 candles
/home/jingbo/temp_store/candle_60_data/GBPUSD/20190301/20190301-GBPUSD-3.pickle
1440
Completed 124/3750 candles
/home/jingbo/temp_store/candle_30_data/GBPUSD/20190220/20190220-GBPUSD-2.pickle
2880
Completed 125/3750 candles
/home/jingbo/temp_store/candle_60_data/EURUSD/20190213/20190213-EURUSD-1.pickle
1440
Completed 126/3750 candles
/home/jingbo/temp_store/candle_600_data/NZDUSD/20190201/20190201-NZDUSD-

288
Completed 186/3750 candles
/home/jingbo/temp_store/candle_600_data/EURUSD/20190212/20190212-EURUSD-3.pickle
144
Completed 187/3750 candles
/home/jingbo/temp_store/candle_300_data/EURUSD/20190221/20190221-EURUSD-3.pickle
288
Completed 188/3750 candles
/home/jingbo/temp_store/candle_300_data/GBPUSD/20190214/20190214-GBPUSD-4.pickle
288
Completed 189/3750 candles
/home/jingbo/temp_store/candle_600_data/EURUSD/20190219/20190219-EURUSD-4.pickle
144
Completed 190/3750 candles
/home/jingbo/temp_store/candle_600_data/GBPUSD/20190208/20190208-GBPUSD-3.pickle
144
Completed 191/3750 candles
/home/jingbo/temp_store/pickled_data/EURUSD/20190227/20190227-EURUSD-4.pickle
Completed 192/3750 candles
/home/jingbo/temp_store/candle_600_data/EURUSD/20190301/20190301-EURUSD-3.pickle
144
/home/jingbo/temp_store/organized_data/EURUSD/20190227/20190227-EURUSD-5.csv
Completed 193/3750 candles
/home/jingbo/temp_store/pickled_data/EURUSD/20190213/20190213-EURUSD-4.pickle
/home/jingbo/temp_store/organized_dat

...

Completed 2952/3750 candles
/home/jingbo/temp_store/candle_60_data/USDCAD/20190227/20190227-USDCAD-4.pickle
1440
Completed 2953/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190204/20190204-USDJPY-2.pickle
288
/home/jingbo/temp_store/candle_10_data/USDJPY/20190221/20190221-USDJPY-1.pickle
8640
Completed 2954/3750 candles
/home/jingbo/temp_store/candle_30_data/USDJPY/20190208/20190208-USDJPY-1.pickle
2880
/home/jingbo/temp_store/candle_10_data/USDJPY/20190211/20190211-USDJPY-2.pickle
8640
Completed 2955/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190206/20190206-USDJPY-1.pickle
Completed 2956/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190227/20190227-USDJPY-5.pickle
/home/jingbo/temp_store/organized_data/USDJPY/20190206/20190206-USDJPY-2.csv
/home/jingbo/temp_store/organized_data/USDJPY/20190228/20190228-USDJPY-1.csv
/home/jingbo/temp_store/candle_10_data/USDJPY/20190215/20190215-USDJPY-2.pickle
8640
Completed 2957/3750 candles
/home/jingbo

2880
Completed 3009/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190206/20190206-USDJPY-5.pickle
Completed 3010/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190211/20190211-USDJPY-5.pickle
144
Completed 3011/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190226/20190226-USDJPY-4.pickle
144
Completed 3012/3750 candles
/home/jingbo/temp_store/candle_30_data/USDJPY/20190222/20190222-USDJPY-5.pickle
2880
Completed 3013/3750 candles
/home/jingbo/temp_store/candle_30_data/USDJPY/20190215/20190215-USDJPY-2.pickle
2880
Completed 3014/3750 candles
/home/jingbo/temp_store/candle_30_data/USDJPY/20190206/20190206-USDJPY-2.pickle
2880
Completed 3015/3750 candles
/home/jingbo/temp_store/candle_60_data/USDJPY/20190208/20190208-USDJPY-1.pickle
1440
Completed 3016/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190218/20190218-USDJPY-4.pickle
288
Completed 3017/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190213/20190213-USDJP

8640
Completed 3073/3750 candles
/home/jingbo/temp_store/candle_60_data/USDJPY/20190228/20190228-USDJPY-3.pickle
1440
Completed 3074/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190225/20190225-USDJPY-3.pickle
144
Completed 3075/3750 candles
/home/jingbo/temp_store/pickled_data/USDCHF/20190228/20190228-USDCHF-3.pickle
Completed 3076/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190222/20190222-USDJPY-1.pickle
/home/jingbo/temp_store/organized_data/USDJPY/20190222/20190222-USDJPY-2.csv
Completed 3077/3750 candles
/home/jingbo/temp_store/candle_30_data/USDCAD/20190228/20190228-USDCAD-2.pickle
2880
Completed 3078/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190205/20190205-USDJPY-1.pickle
/home/jingbo/temp_store/candle_10_data/USDJPY/20190222/20190222-USDJPY-2.pickle
8640
Completed 3079/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190213/20190213-USDJPY-4.pickle
288
Completed 3080/3750 candles
/home/jingbo/temp_store/pickled_dat

1440
Completed 3137/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190215/20190215-USDJPY-2.pickle
144
Completed 3138/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190228/20190228-USDJPY-1.pickle
288
Completed 3139/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190221/20190221-USDJPY-1.pickle
288
Completed 3140/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190204/20190204-USDJPY-3.pickle
288
Completed 3141/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190207/20190207-USDJPY-2.pickle
144
Completed 3142/3750 candles
/home/jingbo/temp_store/candle_30_data/USDJPY/20190221/20190221-USDJPY-4.pickle
2880
Completed 3143/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190219/20190219-USDJPY-2.pickle
288
Completed 3144/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190208/20190208-USDJPY-1.pickle
Completed 3145/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190228/20190228-USDJPY-3

288
Completed 3204/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190301/20190301-USDJPY-2.pickle
144
Completed 3205/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190220/20190220-USDJPY-3.pickle
288
Completed 3206/3750 candles
/home/jingbo/temp_store/candle_60_data/USDJPY/20190227/20190227-USDJPY-3.pickle
1440
Completed 3207/3750 candles
/home/jingbo/temp_store/pickled_data/USDJPY/20190228/20190228-USDJPY-4.pickle
Completed 3208/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190207/20190207-USDJPY-3.pickle
288
Completed 3209/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190214/20190214-USDJPY-2.pickle
144
Completed 3210/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190215/20190215-USDJPY-3.pickle
144
Completed 3211/3750 candles
/home/jingbo/temp_store/candle_600_data/USDJPY/20190220/20190220-USDJPY-3.pickle
144
Completed 3212/3750 candles
/home/jingbo/temp_store/candle_300_data/USDJPY/20190227/20190227-USDJPY

* Load a list of missing files

In [9]:
missing = pickle.load(open('missing_files.pickle', 'rb'))
total_count = len(missing) * len(valid_intervals)
print(f'Total number of missing files: {total_count}')

for m in missing:
    # print(m)
    all_arguments.append((*m, completed_count))

Total number of missing files: 3225


* Run through all data

In [9]:
for day in ALL_DAYS:
    for pair in valid_pairs:
        for lp in range(1, 6):
            all_arguments.append((day, pair, lp, completed_count))

* Now run stuff in parallel

In [10]:
pl.map(generate_for_one_file, all_arguments)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Check Progress

In [19]:
all_pickles = 0
completed_pickles = 0

all_candles = 0
completed_candles = 0

all_arguments = []
for day in ALL_DAYS:
    day = day.replace('-', '')
    for pair in valid_pairs:
        for lp in range(1, 6):
            fname = f'/sailhome/jingbo/CXR_RELATED/temp_store/organized_data/{pair}/{day}/{day}-{pair}-{lp}.csv'
            all_pickles += 1
            if os.path.exists(fname):
                completed_pickles += 1
            
            for interval in valid_intervals:
                cname = f'/sailhome/jingbo/CXR_RELATED/temp_store/candle_{interval}_data/{pair}/{day}/{day}-{pair}-{lp}.pickle'
                all_candles += 1
                
                if os.path.exists(cname):
                    completed_candles += 1

In [20]:
print(f'{completed_pickles}/{all_pickles}')
print(f'{completed_candles}/{all_candles}')

750/750
848/3750


In [ ]:
df = pd.read_csv('/sailhome/jingbo/CXR_RELATED/temp_store/organized_data/AUDUSD/20190201/20190201-AUDUSD-1.csv', parse_dates=['time'])

In [ ]:
candle_start, bid_candles, ask_candles, last_iloc = generate_candles(df, parser.parse('20190201'), datetime.timedelta(seconds=10))

In [ ]:
print(last_iloc)
print(df.iloc[:last_iloc])

In [ ]:
df.iloc[163321]

In [ ]:
candle_df = pd.DataFrame(bid_candles)
candle_df.index = candle_start

In [ ]:
candle_df

In [ ]:
# df.iloc[1000]
df.iloc[10*10000 + 700]

In [ ]:
len(df)

In [ ]:
df.iloc[667243]